필요한 라이브러리 import 및 데이터 가져오기


In [ ]:
!pip install konlpy

In [13]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import sys
import json
import nltk
nltk.download('stopwords')

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer


dev_file = open('friends_test.json', encoding="utf-8")
dev_data = json.load(dev_file)
train_file = open('friends_train.json', encoding="utf-8")
train_data = json.load(train_file)
test_file = open('friends_dev.json', encoding="utf-8")
test_data = json.load(test_file)

df_dev = pd.DataFrame(columns=['annotation', 'emotion', 'speaker', 'utterance'])
df_train = pd.DataFrame(columns=['annotation', 'emotion', 'speaker', 'utterance'])
df_test = pd.DataFrame(columns=['annotation', 'emotion', 'speaker', 'utterance'])

for i in range(len(dev_data)):
    df_dev = pd.concat([df_dev, pd.DataFrame(dev_data[i])])

for i in range(len(train_data)):
    df_train = pd.concat([df_train, pd.DataFrame(train_data[i])])

for i in range(len(test_data)):
    df_test = pd.concat([df_test, pd.DataFrame(test_data[i])])

df_dev = df_dev.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


데이터 진행 과정 확인을 위한 Progress Bar Util

In [14]:
def printProgress (iteration, total, prefix = '', suffix = '', decimals = 1, barLength = 100):
    formatStr = "{0:." + str(decimals) + "f}"
    percent = formatStr.format(100 * (iteration / float(total)))
    filledLength = int(round(barLength * iteration / float(total)))
    bar = '#' * filledLength + '-' * (barLength - filledLength)
    sys.stdout.write('\r%s |%s| %s%s %s' % (prefix, bar, percent, '%', suffix)),
    if iteration == total:
        sys.stdout.write('\n')
    sys.stdout.flush()

영어 데이터 전처리 함수

In [20]:
lancaster_stemmer = LancasterStemmer()
stemmer = SnowballStemmer('english') 
wordnet_lemmatizer = WordNetLemmatizer()

stops = set(stopwords.words('english'))

df_dev['words'] = ''
df_train['words'] = ''
df_test['words'] = ''

def comment_to_words(data):
    # 1. 영어가 아닌 문자는 공백으로 변환
    data = re.sub('[^a-zA-Z]', ' ', data)
    
    # 2. 소문자로 변환
    lowerdata = data.lower()
    
    # 3. 문자열로 변환
    words = lowerdata.split()
    
    # 4. 불용어 제거
    words = [w for w in words if not w in stops]
    
    # 5. 어간추출
    stemming_words = [stemmer.stem(w) for w in words]
    
    # 7. 공백으로 구분된 문자열로 결합하여 결과를 반환
    words = ' '.join(stemming_words)
    return words

훈련 & 개발 & 테스트 데이터 전처리

In [22]:
# 개발 데이터셋 전처리
for i in range(0, len(df_dev)):
    df_dev.loc[i, 'words'] = comment_to_words(df_dev.loc[i, 'utterance'])

# 훈련 데이터셋 전처리
for i in range(0, len(df_train)):
    df_train.loc[i, 'words'] = comment_to_words(df_train.loc[i, 'utterance'])    

# 테스트 데이터셋 전처리
for i in range(0, len(df_test)):
    df_test.loc[i, 'words'] = comment_to_words(df_test.loc[i, 'utterance'])

In [25]:
y_info = [['neutral', 'neutral'],
          ['surprise', 'surprise'],
          ['non-neutral', 'non-neutral'],
          ['joy', 'joy'],
          ['sadness', 'sadness'],
          ['anger', 'anger'],
          ['disgust', 'disgust']]

df_y = pd.DataFrame(y_info, columns=['emotion', 'Y'])

df_dev = pd.merge(df_dev, df_y, on=['emotion'])
df_train = pd.merge(df_train, df_y, on=['emotion'])
df_test = pd.merge(df_test, df_y, on=['emotion'])

Tokenizing

In [28]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import backend as K

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection  import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [29]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000

# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250

# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_dev['words'].values)
word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

Found 1891 unique tokens.


패딩

In [30]:
X = tokenizer.texts_to_sequences(df_train['words'].values)
X = sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (10376, 250)


In [32]:
Y = pd.get_dummies(df_train['Y']).values
print('Shape of label tensor:', Y.shape)


Shape of label tensor: (10376, 7)


In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8300, 250) (8300, 7)
(2076, 250) (2076, 7)


학습

In [43]:
EMBEDDING_DIM = 250
MAX_NB_WORDS = len(X_train)

import keras
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, SpatialDropout1D, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
#model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[0]))
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

history = model.fit(X_train, 
                    Y_train, 
                    epochs=15, 
                    batch_size=60,
                    validation_split=0.2,
                    callbacks=[es, mc])

Epoch 1/15
111/111 [==============================] - 116s 1s/step - loss: 0.3805 - acc: 0.4223 - val_loss: 0.3320 - val_acc: 0.4596

Epoch 00001: val_acc improved from -inf to 0.45964, saving model to best_model.h5
Epoch 2/15
111/111 [==============================] - 113s 1s/step - loss: 0.3325 - acc: 0.4525 - val_loss: 0.3181 - val_acc: 0.4994

Epoch 00002: val_acc improved from 0.45964 to 0.49940, saving model to best_model.h5
Epoch 3/15
111/111 [==============================] - 113s 1s/step - loss: 0.3124 - acc: 0.4920 - val_loss: 0.3185 - val_acc: 0.4747

Epoch 00003: val_acc did not improve from 0.49940
Epoch 4/15
111/111 [==============================] - 114s 1s/step - loss: 0.3016 - acc: 0.5141 - val_loss: 0.3150 - val_acc: 0.4813

Epoch 00004: val_acc did not improve from 0.49940
Epoch 5/15
111/111 [==============================] - 113s 1s/step - loss: 0.2880 - acc: 0.5465 - val_loss: 0.3151 - val_acc: 0.4849

Epoch 00005: val_acc did not improve from 0.49940
Epoch 6/15
11

가장 좋은 모델을 기준으로 테스트 데이터 정확도 측정

In [57]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, Y_test)[1]))

65/65 [==============================] - 7s 99ms/step - loss: 0.3229 - acc: 0.4812

 테스트 정확도: 0.4812


Kaggle Sample 데이터를 통한 결과 분류 시작

In [60]:
data = pd.read_csv("data.csv", sep=",")
data['words'] = ''

print(data[:5])

for i in range(0, len(data)):
    data.loc[i, 'words'] = comment_to_words(data.loc[i, 'utterance']) 

print(data[:5])

X_last = tokenizer.texts_to_sequences(data['words'].values)
X_last = sequence.pad_sequences(X_last, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_last.shape)

predicted = loaded_model.predict(X_last)

print(predict[:5])

cols = ['Id', 'Predicted']
result_df = pd.DataFrame([], columns=cols)

id_list = []
result = []
result2 = []

for predict in predicted:
    result.append([np.argmax(predict), y_info[np.argmax(predict)][0]])
    result2.append( y_info[np.argmax(predict)][0])

for i in range(0, len(data)):
    id_list.append(i)

result_df['Id'] = id_list
result_df['Predicted'] = result2

   id  i_dialog  ...                                          utterance words
0   0         0  ...                      Alright, whadyou do with him?      
1   1         0  ...                                  Oh! You're awake!      
2   2         0  ...  Then you gotta come clean with Ma! This is not...      
3   3         0  ...                                  Yeah, but this is      
4   4         0  ...          I don't wanna hear it! Now go to my room!      

[5 rows x 6 columns]
   id  ...                   words
0   0  ...         alright whadyou
1   1  ...                 oh awak
2   2  ...  gotta come clean right
3   3  ...                    yeah
4   4  ...      wanna hear go room

[5 rows x 6 columns]
Shape of data tensor: (1623, 250)
[0.03803337 0.0232929  0.20965481 0.50287366 0.11513114]


분석된 결과를 CSV 파일로 export

In [55]:
result_df.to_csv('sample.csv', sep=',', index=False)